Machine Learning - Exercise 03 - Naive Bayes Classifier

Sinh viên: Nguyễn Hoàng Lộc

MSSV: 1511849

# Data preparation

In [0]:
import pandas as pd
import numpy as np
import sys
from google.colab import drive
import operator
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from copy import deepcopy as clone
from sklearn.feature_extraction.text import CountVectorizer
eps = np.finfo(float).eps

In [3]:
drive.mount('/content/drive')
sys.path.append('drive/My Drive/Colab Notebooks/') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
articles_path = 'drive/My Drive/Colab Notebooks/ML_Exercises/articles/'
filename_train = [articles_path + "giaoduc" + str(i) + ".txt" for i in range(100)]
filename_train+= [articles_path + "thegioi" + str(i) + ".txt" for i in range(100)]
filename_train+= [articles_path + "phapluat" + str(i) + ".txt" for i in range(100)]
filename_train+= [articles_path + "suckhoe" + str(i) + ".txt" for i in range(100)]

filename_test = [articles_path + "giaoduc" + str(i) + ".txt" for i in range(100,150)]
filename_test+= [articles_path + "thegioi" + str(i) + ".txt" for i in range(100,150)]
filename_test+= [articles_path + "phapluat" + str(i) + ".txt" for i in range(100,150)]
filename_test+= [articles_path + "suckhoe" + str(i) + ".txt" for i in range(100,150)]

In [0]:
X_train_raw = list()
for filename in filename_train:
    X_train_raw.append(open(filename, 'r').read()) 
X_test_raw = list()
for filename in filename_test:
    X_test_raw.append(open(filename, 'r').read())
       
y_train_raw = [0]*100 + [1]*100 + [2]*100 + [3]*100
y_test_raw = [0]*50 + [1]*50 + [2]*50 + [3]*50

countVectorizer = CountVectorizer(max_df=0.5,min_df=2)

In [0]:
X_train = countVectorizer.fit_transform(X_train_raw)
X_train = pd.DataFrame(data = X_train.toarray(), columns = countVectorizer.get_feature_names())
y_train = pd.DataFrame(data = y_train_raw, columns = ['class'])

X_test = countVectorizer.transform(X_test_raw)
X_test  = pd.DataFrame(data = X_test.toarray(), columns = countVectorizer.get_feature_names())
y_test = pd.DataFrame(data = y_test_raw, columns = ['class'])

X_train.drop(X_train.columns[range(0,250)], axis=1,inplace=True)
X_test.drop(X_test.columns[range(0,250)], axis=1,inplace=True)
X_train[X_train>1] = 1
X_test[X_test>1] = 1

# Implementation

In [0]:
class NaiveBayesClassifier():

  def fit(self, X, y):
    self.labels = set(list(y['class']))
    num_instance, num_feature = X.shape #num, D
    self.prior = {label : 0 for label in self.labels} #dict chứa các prior của các nhãn
    self.theta = {label:np.zeros(num_feature) for label in self.labels} #Dict chứa mean của các feature ứng với mỗi label   
    for label in self.labels:
      idx = list(y.loc[y['class'] == label].index.values) #index của các instances với nhãn label
      sub_X = X.iloc[idx,:].values #Lấy ra sub data của X ứng với index vừa tìm
      self.prior[label] = len(sub_X) / num_instance  #Tính P(label)         
      N_jc = np.sum(sub_X, axis = 0) #Số lượng instances có giá trị bằng 1 của từng feature
      self.theta[label] = N_jc / num_instance #Tính means
    
  def predict(self, X):
    num_instance = len(X)
    prior = {label : 0 for label in self.labels}
    logPost = { i: clone(prior) for i in range(num_instance)}
    y_hat = {i:None for i in range(num_instance)} #Chứa label cho dữ liệu test để return
    for index, row in X.iterrows():
      instance = row.values #Mỗi instance là mỗi dòng của dataframe
      for label in self.labels:
        current_theta = self.theta[label]
        loglik = np.sum(instance*np.log(current_theta + eps) + (1-instance)*np.log(1-current_theta + eps)) 
        logPost[index][label] = np.log(prior[label] + eps) + loglik 
      y_hat[index] = max(logPost[index].items(), key = operator.itemgetter(1))[0]
    return list(y_hat.values())

In [0]:
nb_clf = NaiveBayesClassifier()
nb_clf.fit(X_train, y_train)
predicted = nb_clf.predict(X_test)

In [204]:
print('Accuracy: ',accuracy_score(predicted, y_test))
print( classification_report(predicted, y_test))

Accuracy:  0.915
             precision    recall  f1-score   support

          0       0.92      0.90      0.91        51
          1       0.88      1.00      0.94        44
          2       0.88      0.96      0.92        46
          3       0.98      0.83      0.90        59

avg / total       0.92      0.92      0.91       200



#Compare with sklearn 

In [0]:
from sklearn.naive_bayes import BernoulliNB

In [187]:
clf = BernoulliNB()
clf.fit(X_train, y_train)
predicted = clf.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [188]:
print('Accuracy: ',accuracy_score(predicted, y_est))
print( classification_report(predicted, y_test))

Accuracy:  0.94
             precision    recall  f1-score   support

          0       0.94      0.94      0.94        50
          1       1.00      1.00      1.00        50
          2       0.90      0.88      0.89        51
          3       0.92      0.94      0.93        49

avg / total       0.94      0.94      0.94       200

